In [2]:
#take vote format to be ["voter",[-ranking-]]
from numpy import random
import numpy as np
def test_ballot_generator(NVoters,NCandidates):
    ranks=np.array(list(range(1,NCandidates+1)))
    #print(ranks)
    ballots=[]
    for i in range(1,NVoters+1):
        voter="Voter"+str(i)
        ranking=random.permutation(ranks)
        #print(ranking)
        ballots+=[[voter,ranking]]
    return ballots
def count_first_place_votes(candidate,ballots):
    #candidate is an integer here.
    tot=0
    for a in ballots:
        if a[1][candidate]==1:
            tot+=1
    return int(tot)
def count_nth_place_votes(candidate,ballots,n):
    #candidate is an integer here.
    tot=0
    for a in ballots:
        if a[1][candidate]==n:
            tot+=1
    return int(tot)
def balloting_stage(candidates,ballots,tround=1,stage=1,verbose=True):
    total_votes=len(ballots)
    first_place_votes=np.zeros(len(candidates))
    for i in range(len(candidates)):
        first_place_votes[i]=count_first_place_votes(i,ballots)
    if verbose:
        print("balloting round",tround,"stage",stage,": \nfirst place votes:", [(candidates[i],first_place_votes[i]) for i in range(len(candidates))])
    most_votes=0
    #candidate_with_most_votes_index=NaN
    least_votes=sum(first_place_votes)
    #candidate_with_least_votes_index=NaN
    for i in range(len(candidates)):
        if first_place_votes[i]>most_votes:
            most_votes=first_place_votes[i]
            candidate_with_most_votes_index=i
        if first_place_votes[i]<least_votes:
            least_votes=first_place_votes[i]
            candidate_with_least_votes_index=i
    ties_most=[]
    ties_least=[]
    for i in range(len(candidates)):
        if first_place_votes[i]==most_votes:
            ties_most+=[i]
        if first_place_votes[i]==least_votes:
            ties_least+=[i]
    if verbose:
        if len(ties_most)>1:
            print("balloting round",tround,"stage",stage,": \nwarning: tie for first between ",[candidates[i] for i in ties_most])
        if len(ties_least)>1:    
            print("balloting round",tround,"stage",stage,": \nwarning: tie for last between ",[candidates[i] for i in ties_least])
    if ties_least==ties_most:
        if verbose: print("balloting round",tround,"stage",stage,": \nAll way tie")
        return ("tie",list(range(len(candidates))))
    if most_votes>total_votes//2:
        outcome="victory"
        outcome_candidate_indices=[candidate_with_most_votes_index]
        if verbose:
            print("balloting round",tround,"stage",stage,": \n",candidates[candidate_with_most_votes_index],"wins outright with", most_votes,"out of",total_votes)
    else:
        outcome="elimination"
        outcome_candidate_indices=ties_least
        if verbose:
            print("balloting round",tround,"stage",stage,": \n",[candidates[i] for i in ties_least],"are eliminated with", least_votes,"out of",total_votes)
    return (outcome,outcome_candidate_indices)
def ballot_change(ballots,candidate_index,verbose=False,debug=False):
    newballots=[]
    ballot_length=len(ballots[0][1])
    if debug: print("this is ballot_change (before): candidate index",candidate_index,"\nsample ballot:",ballots[0])
    for a in ballots:
        cand_rank=a[1][candidate_index]
        new_ranking=[]
        for i in range(ballot_length):
            if a[1][i]>cand_rank:
                new_ranking+=[a[1][i]-1]
            if a[1][i]<cand_rank:
                new_ranking+=[a[1][i]]
        newballot=[a[0],np.array(new_ranking)]
        if verbose: print("this is ballot_change, adding",newballot)
        newballots+=[newballot]
    if debug: print("this is ballot_change (after): candidate index",candidate_index,"\nsample ballot:",ballots[0])
    return newballots
def candidate_elim(candidates,indices):
    newcands=[]
    for i in range(len(candidates)):
        if i not in indices:
            newcands+=[candidates[i]]
    return newcands
def RCV_recursive(input_candidates,ballots,ranking=[],tround=0,verbose=True,debug=False): #this performs a round, then calls the next round recursively
    if len(input_candidates)==0:
        if verbose: print("voting complete after",tround,"rounds!")
        if verbose: print(ranking)
        return ranking
    tround+=1
    if len(input_candidates)==1:
        if verbose: print("voting complete after",tround,"rounds!")
        ranking+=[(input_candidates[0],tround)]
        if verbose: print(ranking)
        return ranking
    if debug: print("\n\n!~!start of overall round",tround,"\n!~!current candidates are",input_candidates,"\n!~!current ranking is", ranking)
    candidates=[(input_candidates[i],i) for i in range(len(input_candidates))]
    outcome="initiated"
    elim_indices=[]
    newballots=ballots
    newcandidates=candidates
    stage=0
    #stagecandidates=candidates
        #tround+=1
        #roundcandidates=candidate_elim(newcandidates)
    while outcome not in ["victory","tie"]: #perform stage of round
        stage+=1
        if debug: print("\nstage",stage,"checkpoint ")
        for i in elim_indices[::-1]:
            newballots=ballot_change(newballots,i,False,debug)
        if debug: print("stage",stage,"sample ballot",newballots[0])
        newcandidates=candidate_elim(newcandidates,elim_indices)
        if debug: print("stage",stage,"new candidates are:",newcandidates)
        outcome,elim_indices=balloting_stage(newcandidates,newballots,tround,stage,verbose)
        if debug: print(outcome,elim_indices)
    if len(elim_indices)>1:
        ranking+=[(candidates[i][0],tround)for i in elim_indices]
    else:
        winner=candidates[elim_indices[0]][0]
        ranking+=[(winner,tround)]
    candidate_elim_indices=[candidates[i][1] for i in elim_indices]
    next_input_candidates=candidate_elim(input_candidates,candidate_elim_indices)
    if debug: print("next round candidates:",next_input_candidates)
    next_ballots=ballots
    for i in candidate_elim_indices[::-1]:
        next_ballots=ballot_change(next_ballots,i,False,debug)
    if debug: print("next round sample ballot:",next_ballots[0])
    if debug: print("ranking after round",tround,":\n",ranking)
    return RCV_recursive(next_input_candidates,next_ballots,ranking,tround,verbose,debug)
def RCV_wrapper(candidates,ballots,verbosei=True,debugi=False):
    ranking=RCV_recursive(candidates,ballots,ranking=[],tround=0,verbose=verbosei,debug=debugi)
    clean_ranking=[]
    nrounds=ranking[-1][1]
    for i in range(1,nrounds+1):
        roundlist=[]
        for a in ranking:
            if a[1]==i:
                roundlist+=[a[0]]
        if len(roundlist)>1:
            clean_ranking+=[roundlist]
        else:
            clean_ranking+=[roundlist[0]]
    return clean_ranking
def RCV_vecs(candidates,ballots,debug=False):
    candvecs=[]
    for i in range(len(candidates)):
        candvotes=[]
        for b in ballots:
            candvotes+=b[1][i]
        candvecs+=[[candidates[i],candvec]]
    return candvecs
def RCV_tots(candidates,ballots,debug=False):
    k=len(candidates)
    totlist=[]
    for i in range(k):
        tot_i_list=[]
        for n in range(k):
            m=count_nth_place_votes(i,ballots,n+1)
            tot_i_list+=[m]
        totlist+=[tot_i_list]
    return totlist
def first_nonzero(L):
    l=len(L)
    for i in range(l):
        if L[i]!=0:
            return i
    return l
def last_nonzero(L):
    l=len(L)
    for i in range(l):
        if L[-i-1]!=0:
            return l-(i+1)
    return l
def RCV_stats(candidates,ballots,debug=False):
    tots=RCV_tots(candidates,ballots,debug)
    if debug: print(tots)
    k=len(candidates)
    stats=[[a] for a in candidates]
    for i in range(k):
        if debug: print(tots[i])
        avg_i=sum((j+1)*tots[i][j] for j in range(k))/sum(tots[i][j] for j in range(k))
        best_vote=first_nonzero(tots[i])
        worst_vote=last_nonzero(tots[i])
        num_best=tots[i][best_vote]
        num_worst=tots[i][worst_vote]
        stats[i]+=[avg_i,(best_vote+1,num_best),(worst_vote+1,num_worst)]
    return stats
        
            
def MC_sorter(tab):
    tab_sorted=[]
    tab_unsorted=tab
    #print("this is MC_sorter, sorting", tab_unsorted)
    while len(tab_unsorted)>0:
        most_votes=-1
        for i in range(len(tab_unsorted)):
            if tab_unsorted[i][1]>most_votes:
                most_votes=tab_unsorted[i][1]
                greatdex=i
        tab_sorted+=[tab_unsorted[greatdex]]
        tab_unsorted=tab_unsorted[:greatdex]+tab_unsorted[greatdex+1:]
    return tab_sorted
def sort_by_index(tab,j):
    tab_sorted=[]
    tab_unsorted=tab
    #print("this is MC_sorter, sorting", tab_unsorted)
    while len(tab_unsorted)>0:
        most_votes=-1
        for i in range(len(tab_unsorted)):
            if tab_unsorted[i][j]>most_votes:
                most_votes=tab_unsorted[i][j]
                greatdex=i
        tab_sorted+=[tab_unsorted[greatdex]]
        tab_unsorted=tab_unsorted[:greatdex]+tab_unsorted[greatdex+1:]
    return tab_sorted     
        
    
def MC_tabulator(choices,ballots):
    tabulation={a:0 for a in choices}
    for b in ballots:
        for a in choices:
            if b[1][0]==a:
                tabulation[a]+=1
    tab_unsorted=[(a,tabulation[a]) for a in choices]
    return MC_sorter(tab_unsorted)
def test_MC_generator(choices,N):
    m=len(choices)
    ballots=[]
    for i in range(N):
        k=random.randint(m)
        ballots+=[["voter "+str(i),[choices[k]]]]
    return ballots

In [3]:
choices=["lynx","mystics","dream","fever","aces","liberty","sun","mercury"]
ballots=test_MC_generator(choices,30)
print(ballots)
MC_tabulator(choices,ballots)
    

[['voter 0', ['sun']], ['voter 1', ['mercury']], ['voter 2', ['sun']], ['voter 3', ['fever']], ['voter 4', ['mystics']], ['voter 5', ['sun']], ['voter 6', ['lynx']], ['voter 7', ['sun']], ['voter 8', ['lynx']], ['voter 9', ['sun']], ['voter 10', ['fever']], ['voter 11', ['aces']], ['voter 12', ['aces']], ['voter 13', ['fever']], ['voter 14', ['fever']], ['voter 15', ['mystics']], ['voter 16', ['lynx']], ['voter 17', ['aces']], ['voter 18', ['dream']], ['voter 19', ['lynx']], ['voter 20', ['sun']], ['voter 21', ['aces']], ['voter 22', ['mercury']], ['voter 23', ['lynx']], ['voter 24', ['liberty']], ['voter 25', ['fever']], ['voter 26', ['liberty']], ['voter 27', ['aces']], ['voter 28', ['mercury']], ['voter 29', ['mercury']]]


[('sun', 6),
 ('lynx', 5),
 ('fever', 5),
 ('aces', 5),
 ('mercury', 4),
 ('mystics', 2),
 ('liberty', 2),
 ('dream', 1)]

In [4]:
candidates=["Laces", "Owls","Mist","Phantom","Rose","Vinyl"]
ballots=test_ballot_generator(50,6)
#print(ballots)
stats=RCV_stats(candidates,ballots)
print(stats)


[['Laces', 3.64, (1, 5), (6, 9)], ['Owls', 3.6, (1, 9), (6, 12)], ['Mist', 3.46, (1, 9), (6, 8)], ['Phantom', 3.4, (1, 9), (6, 5)], ['Rose', 3.4, (1, 9), (6, 8)], ['Vinyl', 3.5, (1, 9), (6, 8)]]


In [5]:
candidates=["Laces", "Owls","Mist","Phantom","Rose","Vinyl"]
completed_trials=0
for i in range(30):
    print("\n\n\n","_"*15,"\nTRIAL NUMBER",i,"\n","_"*15,"\n")
    ballots=test_ballot_generator(50,6)
    ranks=RCV_wrapper(candidates,ballots,debugi=True)
    print(ranks)
    if len(ranks)==len(candidates):
        completed_trials+=1
print("\n\nCOMPLETED TRIALS:",completed_trials)




 _______________ 
TRIAL NUMBER 0 
 _______________ 



!~!start of overall round 1 
!~!current candidates are ['Laces', 'Owls', 'Mist', 'Phantom', 'Rose', 'Vinyl'] 
!~!current ranking is []

stage 1 checkpoint 
stage 1 sample ballot ['Voter1', array([4, 2, 3, 1, 6, 5])]
stage 1 new candidates are: [('Laces', 0), ('Owls', 1), ('Mist', 2), ('Phantom', 3), ('Rose', 4), ('Vinyl', 5)]
balloting round 1 stage 1 : 
first place votes: [(('Laces', 0), 8.0), (('Owls', 1), 11.0), (('Mist', 2), 8.0), (('Phantom', 3), 6.0), (('Rose', 4), 9.0), (('Vinyl', 5), 8.0)]
balloting round 1 stage 1 : 
 [('Phantom', 3)] are eliminated with 6.0 out of 50
elimination [3]

stage 2 checkpoint 
this is ballot_change (before): candidate index 3 
sample ballot: ['Voter1', array([4, 2, 3, 1, 6, 5])]
this is ballot_change (after): candidate index 3 
sample ballot: ['Voter1', array([4, 2, 3, 1, 6, 5])]
stage 2 sample ballot ['Voter1', array([3, 1, 2, 5, 4])]
stage 2 new candidates are: [('Laces', 0), ('Owls', 1), (

### 

In [12]:
import csv
def vote_parser(input_csv):
    votes=[]
    with open(input_csv,newline='') as csvfile:
        votereader=csv.reader(csvfile,delimiter=',',quotechar='|')
        for row in votereader:
            vote=[str(row[0]),[a for a in row[1:]]]
            votes+=[vote]
        for i in range(len(votes)):
            votes[i][0]="Voter "+str(i+1)
    return votes
def ballot_gen(Questions,ballots,debug=False):
    #Question format: question text (str), column start, column end, method,candidates
    Question_set=[]
    for a in Questions:
        a_ballots=[]
        for b in ballots:
            if debug: print(a[0],": ballot indices",a[1]-2,a[2]-1,"ballot length",len(b[1]))
            b_ballot=[b[0]]+[np.array([b[1][i] for i in range(a[1]-2,a[2]-1)])]
            a_ballots+=[b_ballot]
        if len(a)==4:
            Question_set+=[(a[0],a_ballots,a[3])]
        if len(a)==5:
            Question_set+=[(a[0],a_ballots,a[3],a[4])]
    return Question_set
def vote_counter(Question_set,debug=False):
    if debug: print(Question_set)
    processed_questions=[]
    for a in Question_set:
        if a[2]=="RCV":
            for i in a[1]:
                i[1]=i[1].astype(int)
            if debug: print("RCV initiated.",a[1],a[3])    
            rankings=RCV_wrapper(a[3],a[1])
            processed_question=(a[0],rankings)
        elif a[2]=="Linear":
            if debug: print("Linear counting initiated")
            votes=[]
            for b in a[1]:
                if int(b[1][0])>0:
                    votes+=[int(b[1][0])]
            total_cast=sum(1 for v in votes)
            avg=sum(votes)/total_cast
            processed_question=(a[0],avg)
        elif a[2]=="MC":
            ranking=MC_tabulator(a[3],a[1])
            processed_question=(a[0],ranking)
        else:
            processed_question=(a[0],"not implemented yet")
        processed_questions+=[processed_question]
    return processed_questions
#RCV stats format: name, avg ranking, best ranking, #best, worst ranking, #worst
def election_statistics(Question_set,debug=False):
    #todo: integrate this into the rest
    completed_stats=[]
    for a in Question_set:
        if a[2]=="Linear":
            stats=linear_stats(a[1])
            
        elif a[2]=="RCV":
            for i in a[1]:
                i[1]=i[1].astype(int)
            stats=RCV_stats(a[3],a[1],debug)
        else:
            stats="not implemented"
        completed_stats+=[(a[0],stats)]
    return completed_stats
            
def election_doer(input_csv,Qlist,debug=False):
    votes=vote_parser(input_csv)
    if debug: print(votes)
    Question_set=ballot_gen(Qlist,votes)
    if debug: print(Question_set)
    processed_questions=vote_counter(Question_set,debug)
    stats=election_statistics(Question_set,debug)
    return processed_questions,stats

            

In [7]:
def linear_stats(ballots,debug=False):
    votes_cast=[]
    for b in ballots:
        vote=int(b[1][0])
        votes_cast+=[vote]
    voted_options=set(votes_cast)
    stats=[[a,0] for a in voted_options]
    for s in stats:
        for v in votes_cast:
            if v==s[0]:
                s[1]+=1
    stats=sort_by_index(stats,0)
    return stats

In [8]:
A=np.array(['1','2'])

In [9]:
ballots=vote_parser('test_data.csv')

In [10]:
teams=["Laces", "Owls","Mist","Phantom","Rose","Vinyl"]
Week1games=["Mist-Owls","Rose-Vinyl","Phantom-Laces","Owls-Rose","Vinyl-Phantom","Laces-Mist"]
Week2games=["Phantom-Mist","Laces-Vinyl","Mist-Rose","Owls-Phantom","Vinyl-Owls","Rose-Laces"]
Questions_1=[("How are you enjoying Unrivaled so far?",2,2,"Linear"),
             ("Rank the six teams in order of which you think is strongest overall.",3,8,"RCV",teams),
             ("Rank the six teams in order of which you think played best this weekend",9,14,"RCV",teams),
             ("Rank the six teams in order of which you are rooting for.",15,20,"RCV",teams),
             ("Which game was your favorite of the weekend past?",21,21,"MC",Week1games),
             ("Which game are you looking forward to most this coming weekend?",22,22,"MC",Week2games)
            ]
election_doer('week1_data.csv',Questions_1,True)

[['Voter 1', ['8', '2', '1', '5', '6', '4', '3', '1', '2', '5', '6', '4', '3', '2', '1', '5', '6', '4', '3', 'Mist-Owls', 'Laces-Vinyl', '']], ['Voter 2', ['9', '2', '1', '3', '5', '6', '4', '1', '2', '4', '6', '5', '3', '2', '1', '6', '3', '5', '4', 'Mist-Owls', 'Vinyl-Owls', "Phee and Stewie deserve all the flowers and more for what they've done with this."]], ['Voter 3', ['10', '2', '1', '5', '4', '6', '3', '2', '1', '4', '3', '6', '5', '2', '1', '3', '4', '5', '6', 'Mist-Owls', 'Vinyl-Owls', '']], ['Voter 4', ['7', '1', '2', '4', '6', '5', '3', '1', '3', '5', '6', '4', '2', '3', '2', '1', '6', '5', '4', 'Mist-Owls', 'Vinyl-Owls', '']], ['Voter 5', ['9', '1', '2', '4', '6', '5', '3', '1', '2', '3', '6', '5', '4', '2', '1', '3', '6', '4', '5', 'Mist-Owls', 'Vinyl-Owls', '']], ['Voter 6', ['9', '1', '2', '4', '6', '5', '3', '1', '2', '5', '6', '4', '3', '1', '2', '6', '5', '4', '3', 'Mist-Owls', 'Laces-Vinyl', '']], ['Voter 7', ['9', '2', '1', '4', '6', '5', '3', '2', '1', '4', '6', '

([('How are you enjoying Unrivaled so far?', 8.727272727272727),
  ('Rank the six teams in order of which you think is strongest overall.',
   ['Owls', 'Laces', 'Vinyl', 'Mist', 'Rose', 'Phantom']),
  ('Rank the six teams in order of which you think played best this weekend',
   ['Laces', 'Owls', 'Vinyl', 'Mist', 'Rose', 'Phantom']),
  ('Rank the six teams in order of which you are rooting for.',
   ['Owls', 'Laces', 'Vinyl', 'Phantom', 'Rose', 'Mist']),
  ('Which game was your favorite of the weekend past?',
   [('Mist-Owls', 10),
    ('Laces-Mist', 1),
    ('Rose-Vinyl', 0),
    ('Phantom-Laces', 0),
    ('Owls-Rose', 0),
    ('Vinyl-Phantom', 0)]),
  ('Which game are you looking forward to most this coming weekend?',
   [('Laces-Vinyl', 6),
    ('Vinyl-Owls', 4),
    ('Phantom-Mist', 0),
    ('Mist-Rose', 0),
    ('Owls-Phantom', 0),
    ('Rose-Laces', 0)])],
 [('How are you enjoying Unrivaled so far?',
   [[10, 2], [9, 5], [8, 3], [7, 1]]),
  ('Rank the six teams in order of which 

In [11]:
election_doer('week1_data.csv',Questions_1,True)

[['Voter 1', ['8', '2', '1', '5', '6', '4', '3', '1', '2', '5', '6', '4', '3', '2', '1', '5', '6', '4', '3', 'Mist-Owls', 'Laces-Vinyl', '']], ['Voter 2', ['9', '2', '1', '3', '5', '6', '4', '1', '2', '4', '6', '5', '3', '2', '1', '6', '3', '5', '4', 'Mist-Owls', 'Vinyl-Owls', "Phee and Stewie deserve all the flowers and more for what they've done with this."]], ['Voter 3', ['10', '2', '1', '5', '4', '6', '3', '2', '1', '4', '3', '6', '5', '2', '1', '3', '4', '5', '6', 'Mist-Owls', 'Vinyl-Owls', '']], ['Voter 4', ['7', '1', '2', '4', '6', '5', '3', '1', '3', '5', '6', '4', '2', '3', '2', '1', '6', '5', '4', 'Mist-Owls', 'Vinyl-Owls', '']], ['Voter 5', ['9', '1', '2', '4', '6', '5', '3', '1', '2', '3', '6', '5', '4', '2', '1', '3', '6', '4', '5', 'Mist-Owls', 'Vinyl-Owls', '']], ['Voter 6', ['9', '1', '2', '4', '6', '5', '3', '1', '2', '5', '6', '4', '3', '1', '2', '6', '5', '4', '3', 'Mist-Owls', 'Laces-Vinyl', '']], ['Voter 7', ['9', '2', '1', '4', '6', '5', '3', '2', '1', '4', '6', '

([('How are you enjoying Unrivaled so far?', 8.727272727272727),
  ('Rank the six teams in order of which you think is strongest overall.',
   ['Owls', 'Laces', 'Vinyl', 'Mist', 'Rose', 'Phantom']),
  ('Rank the six teams in order of which you think played best this weekend',
   ['Laces', 'Owls', 'Vinyl', 'Mist', 'Rose', 'Phantom']),
  ('Rank the six teams in order of which you are rooting for.',
   ['Owls', 'Laces', 'Vinyl', 'Phantom', 'Rose', 'Mist']),
  ('Which game was your favorite of the weekend past?',
   [('Mist-Owls', 10),
    ('Laces-Mist', 1),
    ('Rose-Vinyl', 0),
    ('Phantom-Laces', 0),
    ('Owls-Rose', 0),
    ('Vinyl-Phantom', 0)]),
  ('Which game are you looking forward to most this coming weekend?',
   [('Laces-Vinyl', 6),
    ('Vinyl-Owls', 4),
    ('Phantom-Mist', 0),
    ('Mist-Rose', 0),
    ('Owls-Phantom', 0),
    ('Rose-Laces', 0)])],
 [('How are you enjoying Unrivaled so far?',
   [[10, 2], [9, 5], [8, 3], [7, 1]]),
  ('Rank the six teams in order of which 

In [15]:
teams=["Laces", "Owls","Mist","Phantom","Rose","Vinyl"]
Week1games=["Mist-Owls","Rose-Vinyl","Phantom-Laces","Owls-Rose","Vinyl-Phantom","Laces-Mist"]
Week2games=["Phantom-Mist","Laces-Vinyl","Mist-Rose","Owls-Phantom","Vinyl-Owls","Rose-Laces"]
Week3games=["Phantom-Rose","Laces-Owls","Mist-Vinyl","Rose-Laces","Mist-Phantom","Owls-Vinyl"]
Questions_1=[("How are you enjoying Unrivaled so far?",2,2,"Linear"),
             ("How did you enjoy this week's slate of games?",3,3,"Linear"),
             ("Rank the six teams in order of which you think is strongest overall.",4,9,"RCV",teams),
             ("Rank the six teams in order of which you think played best this weekend",10,15,"RCV",teams),
             ("Rank the six teams in order of which you are rooting for.",16,21,"RCV",teams),
             ("Which game was your favorite of the weekend past?",22,22,"MC",Week2games),
             ("Which game are you looking forward to most this coming weekend?",23,23,"MC",Week3games)
            ]
election_doer('week2.csv',Questions_1,True)

[['Voter 1', ['8', '7', '2', '1', '5', '6', '4', '3', '2', '1', '6', '5', '3', '4', '2', '1', '6', '5', '3', '4', 'Owls-Phantom', 'Owls-Vinyl', '', 'Napheesa Collier']], ['Voter 2', ['10', '10', '1', '2', '6', '4', '5', '3', '2', '1', '6', '4', '5', '3', '1', '2', '6', '4', '5', '3', 'Laces-Vinyl', 'Laces-Owls', 'Truly one of the only sources of joy in this political shitstorm. Love watching our Lynx players dominate', '"Kmac', ' Phee"']], ['Voter 3', ['10', '10', '2', '1', '6', '5', '4', '3', '2', '1', '6', '5', '3', '4', '2', '1', '3', '4', '6', '5', 'Owls-Phantom', 'Laces-Owls', '', 'Courtney Williams']], ['Voter 4', ['9', '9', '2', '1', '4', '6', '5', '3', '1', '2', '3', '6', '5', '4', '2', '1', '3', '6', '4', '5', 'Vinyl-Owls', 'Laces-Owls', '', '"Phee', ' Kmac', ' Tiffany Hayes"']], ['Voter 5', ['8', '8', '2', '1', '6', '5', '4', '3', '2', '1', '6', '5', '3', '4', '2', '1', '6', '5', '4', '3', 'Owls-Phantom', 'Laces-Owls', '', '']], ['Voter 6', ['10', '10', '2', '1', '4', '6', '5

([('How are you enjoying Unrivaled so far?', 9.285714285714286),
  ("How did you enjoy this week's slate of games?", 8.714285714285714),
  ('Rank the six teams in order of which you think is strongest overall.',
   ['Owls', 'Laces', 'Vinyl', 'Rose', 'Phantom', 'Mist']),
  ('Rank the six teams in order of which you think played best this weekend',
   ['Owls', 'Laces', 'Rose', 'Vinyl', 'Phantom', 'Mist']),
  ('Rank the six teams in order of which you are rooting for.',
   ['Owls', 'Laces', 'Phantom', 'Vinyl', 'Rose', 'Mist']),
  ('Which game was your favorite of the weekend past?',
   [('Owls-Phantom', 3),
    ('Vinyl-Owls', 3),
    ('Laces-Vinyl', 1),
    ('Phantom-Mist', 0),
    ('Mist-Rose', 0),
    ('Rose-Laces', 0)]),
  ('Which game are you looking forward to most this coming weekend?',
   [('Laces-Owls', 6),
    ('Owls-Vinyl', 1),
    ('Phantom-Rose', 0),
    ('Mist-Vinyl', 0),
    ('Rose-Laces', 0),
    ('Mist-Phantom', 0)])],
 [('How are you enjoying Unrivaled so far?', [[10, 4],